In [1]:
import pymysql
import pandas as pd
import datetime
from selenium import webdriver
import re
import sys
import os
import time
from  openpyxl import Workbook
from openpyxl.styles import Font,Alignment
import requests
from bs4 import BeautifulSoup
import urllib
from fontTools.ttLib import TTFont
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 

In [2]:
#这里调用数据爬虫获得数据
def flatten(input_list):
    output_list = []
    while True:
        if input_list == []:
            break
        for index, i in enumerate(input_list):

            if type(i) == list:
                input_list = i + input_list[index + 1:]
                break
            else:
                output_list.append(i)
                input_list.pop(index)
                break
    return output_list
#整合数据用的函数
def zhenghe(a,b):#用于数据整合，避免使用numpy
    re=[]
    for x in a:
        re.append(x)
    for x in b:
        re.append(x)
    return re  
def get_info_yingjiesheng():
    print('开始抓取实习信息......\n')
    #加载启动项
    try:
        option = webdriver.ChromeOptions()
        option.add_argument('headless')
        #import win32com
        #应届生求职网的信息
        url = 'http://www.yingjiesheng.com/commend-parttime-1.html'
        wb_data = requests.get(url)
        soup = BeautifulSoup(wb_data.content, 'html.parser', from_encoding='utf-8')
        a = []
        time=[]
        result = []
        for link in soup.select('#mainNav > div.jobList > table > tr > td > a'):
                                #mainNav > div.jobList > table > tbody > tr:nth-child(22) > td.date
            a.append(str(link))
        for link in soup.select('#mainNav > div.jobList > table > tr > td.date'):
                                #mainNav > div.jobList > table > tbody > tr:nth-child(22) > td.date
            time.append(str(link))

        for i in range(len(a)):
            result.append([re.findall(r'<a href="(.*?)" target=',a[i]),
                           re.findall(r'#008000;">(.*?)</span>',a[i]),
                           re.findall(r'</span>(.*?)</a>',a[i]),
                           re.findall(r'<td class="date">(.*?)</td>',time[i])[0]])
        for i in range(len(result)):
            if result[i][-1][-5:]!=datetime.datetime.now().strftime('%m-%d'):
                result[i]=[]
        result=[x[1]+x[2]+x[0] for x in result if x!=[]]
        for i in range(len(result)):
            if len(result[i])!=3:
                result[i]=[]
        result=[x for x in result if x!=[]]
        for x in result:
            x[1]=x[1].split("招聘")
            if len(x[1])==1:
                x[1].append('实习生')
            elif len(x[1])!=2:
                del x[1][1]
                if x[1][1]=='':
                    x[1][1]='实习生'
            elif x[1][1]=='':
                x[1][1]='实习生'
            elif len(x[1])==1:
                x[1].append('实习生')
        result = [flatten(x) for x in result if x!=[]]
        for x in result:
            x[1]=x[1].replace('实习生','')
            x[1]=x[1].replace('2020','')
            x[1]=x[1].replace('春季','')
            x[1]=x[1].replace('秋季','')
            x[1]=x[1].replace('暑假','')
            x[1]=x[1].replace('寒假','')
            x[1]=x[1].replace('暑期','')
            x[1]=x[1].replace('冬季','')
            if '<' in x[1] and '>' in x[1]:
                x[1]=x[1].split('</span>')[1]
        for i in range(len(result)):
                if 'http' not in result[i][-1]:
                    result[i][-1]='http://www.yingjiesheng.com/'+result[i][-1]
        for x in result:
            x.append('应届生求职网',datetime.datetime.now().strftime('%Y-%m-%d'))
        return result    
    except IndexError:
        return []
# 应届生求职网的备用函数
def getinfo_new4(url):
    a=[]
    wb_data = requests.get(url)
    url=[]
    soup = BeautifulSoup(wb_data.content, 'html.parser', from_encoding='utf-8')
    try:
        for link in soup.select('#wrap > div.clear > div.rec.recr > ul'):
                                ##wrap > div.clear > div.rec.recr > ul > li:nth-child(1)
            a.append(link)
            url.append(str(link))
        #print(len(a),len(str(a[0]).split('实习生'))>=4)
        #print(a)
        if len(str(a[0]).split('实习生'))<=4 :
            a=[]
            url=[]
            for link in soup.select('#mainNav > div.recommend.s_clear > div.box.floatr > ul:nth-child(8)'):
                a.append(link)
                url.append(str(link))
        if len(str(a[0]).split('实习生'))<=4:
            a=[]
            url=[]
            for link in soup.select('#mainNav > div.recommend.s_clear > div.box.floatr > ul:nth-child(6)'):
                a.append(link)
                url.append(str(link))
        a = str(a[0]).split('</li>\n<li>\n')
        a = flatten([x.split('li>\n<li') for x in a])
        a=[x for x in a if '.'+datetime.datetime.now().strftime('%d') in x]
           #datetime.datetime.now().strftime('%d')
#         #url=[re.findall(r'href="(.*?)" target=',str(x))[0] for x in a if x!='']
        a=[[re.findall(r'#008000;">(.*?)</span>',str(x)),re.findall(r'</span>(.*?)</a>',str(x)),re.findall(r'href="(.*?)" target=',str(x))] for x in a] 
        info = [flatten(a[i]) for i in range(len(a))]
        for i in range(len(info)):
            if len(info[i])<3:
                info[i]=[]
        info = [x for x in info if x!=[]]
        for i in range(len(info)):
                if 'http' not in info[i][-1]:
                    info[i][-1]='http://www.yingjiesheng.com/'+info[i][-1]

        for x in info:
            x[1]=x[1].split("招聘")
            if len(x[1])==1:
                x[1].append('实习生')
            elif len(x[1])!=2:
                del x[1][1]
                if x[1][1]=='':
                    x[1][1]='实习生'
            elif x[1][1]=='':
                x[1][1]='实习生'
            elif len(x[1])==1:
                x[1].append('实习生')
        info = [flatten(x) for x in info if x!=[]]
        info = [x[:3]+[x[-1],'应届生',datetime.datetime.now().strftime('%Y-%m-%d')] for x in info if x!=[]]
        for x in info:
            x[1]=x[1].replace('实习生','')
            x[1]=x[1].replace('2020','')
            x[1]=x[1].replace('2021','')
            x[1]=x[1].replace('春季','')
            x[1]=x[1].replace('秋季','')
            x[1]=x[1].replace('暑假','')
            x[1]=x[1].replace('寒假','')
            x[1]=x[1].replace('暑期','')
            x[1]=x[1].replace('冬季','')
        return info
    except:
        return []

#获取实习僧信息的爬虫函数
def shuju_shixisen(page,browser,url):
    if url !='no_url':
        browser.get(url.format(page))
    if page==1:
        ttf = []
        ttf_test=[]
        pwd = browser.find_elements_by_xpath('/html/head/style[1]')
        for x in pwd:
            ttf_test=[x.get_attribute('outerHTML')]
            ttf.append(ttf_test[0].split('url(')[1].split(');}<')[0])
        url='https://www.shixiseng.com'
        urllib.request.urlretrieve(url+ttf[0], "shixi.ttf")
    elif os.path.exists('shixi.ttf'):
        pass
    #获得ttf文件并解码
    else:
        ttf = []
        ttf_test=[]
        pwd = browser.find_elements_by_xpath('/html/head/style[1]')
        for x in pwd:
            ttf_test=[x.get_attribute('outerHTML')]
            ttf.append(ttf_test[0].split('url(')[1].split(');}<')[0])
        url='https://www.shixiseng.com'
        urllib.request.urlretrieve(url+ttf[0], "shixi.ttf")
    font = TTFont('shixi.ttf')
    # 由于实习增的这个文件页面刷新前后是不变的，所以不用前后进行字体文件的比对了
    font_base_order = font.getGlyphOrder()[2:]# 下载下来的文件头两个是空的
    # 新下载的问件与原文件进行比对
    # 前10个是0到9，从本地将对应的文字写出来
    map_list =[
        *[str(i) for i in range(10)], u'一', u'师', 'X', u'会', u'四', u'计', u'财', u'场', 'D', 'H',
        'L', 'P', 'T', u'聘', u'招', u'工', 'd', u'周', 'I', u'端', 'p', u'年', 'h', 'x', u'设', u'程',
        u'二', u'五', u'天', 't', 'C', 'G', u'前', 'K', 'O', u'网', 'S', 'W', 'c', 'g', 'k', 'o', 's',
        'w', u'广', u'市', u'月', u'个', 'B', 'F', u'告', 'N', 'R', 'V', 'Z', u'作', 'b', 'f', 'j', 'n',
        'r', 'v', 'z', u'三', u'互', u'生', u'人', u'政', 'A', 'J', 'E', 'I', u'件', 'M', '行', 'Q', 'U',
        'Y', 'a', 'e', 'i', 'm', u'软', 'q', 'u', u'银', 'y', u'联', 
    ]
    # 你会发现网页中编码对应的是font.getBestCmap()的key的16进制的值
    map_dict = {value: '&#' + hex(key)[1:]
                for key, value in font.getBestCmap().items()}
    # 将固定的字体顺序和uni编码进行一一对应，并从map_dict中寻找16进制的值对应的字体
    temp_dict = {map_dict[key]: value for key, value in zip(font_base_order, map_list)}

    #地区
    dq=[]
    dq_test=[]
    pwd = browser.find_elements_by_xpath('//*[@id="__layout"]/div/div[2]/div[2]/div[1]/div[1]/div[1]/div/div[1]/div[1]/p[2]/span[1]')
    for x in pwd:
        dq_test=[x.text,x.get_attribute('href')]
        dq.append(dq_test[0])
    #公司名称
    gs=[]
    gs_test=[]
    pwd = browser.find_elements_by_xpath('//*[@id="__layout"]/div/div[2]/div[2]/div[1]/div[1]/div[1]/div/div[1]/div[2]/p[1]/a')
    for x in pwd:
        gs_test=[x.text,x.get_attribute('href')]
        gs.append(gs_test[0])
    #职位
    zw=[]
    zw_test=[]
    pwd = browser.find_elements_by_xpath('//*[@id="__layout"]/div/div[2]/div[2]/div[1]/div[1]/div[1]/div/div[1]/div[1]/p[1]/a')
    for x in pwd:
        zw_test=[x.get_attribute('outerHTML')]
        m=str(zw_test[0]).split('title="')[1].split('" target=')[0]
        m=''.join(m.split('amp;'))
        for k,value in temp_dict.items():
            m = m.replace(k,value)
        zw.append(m)
    #链接
    xq = []
    xq_test=[]
    pwd = browser.find_elements_by_xpath('//*[@id="__layout"]/div/div[2]/div[2]/div[1]/div[1]/div[1]/div/div[1]/div[1]/p[1]/a')
    for x in pwd:
        xq_test=[x.get_attribute("outerHTML")]
        xq.append(re.findall(r'href="(.*?)" title="',str(xq_test[0]))[0])
    if len(dq) != len(zw) or len(dq) != len(gs) or  len(gs) != len(zw) or len(xq) != len(zw):
        return []
    else:
        return [[dq[i],gs[i],zw[i],xq[i],'实习僧',datetime.datetime.now().strftime('%Y-%m-%d')] for i in range(len(xq))]  


In [3]:
#实习信息的主函数  
path = os.path.abspath(os.curdir)
sys.path.append(str(path))
option = webdriver.ChromeOptions()
option.add_argument('headless')
browser = webdriver.Chrome(r'D:\Chrome\Application\chromedriver.exe', options=option)
browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
result_yingjiesheng=get_info_yingjiesheng()
if result_yingjiesheng==[]:
    result_yingjiesheng=getinfo_new4(r'http://www.yingjiesheng.com')
if result_yingjiesheng==[]:
    print('扎心了!应届生求职网的信息还没有更新......\n')
url_quanguo = "https://www.shixiseng.com/interns?page={}&keyword=&type=intern&area=&months=&days=&degree=&official=&enterprise=IT300&salary=-0&publishTime=day&sortType=zj&city=全国&internExtend="
url_wuhan = "https://www.shixiseng.com/interns?page=1&keyword=&type=intern&area=&months=&days=&degree=&official=&enterprise=&salary=-0&publishTime=day&sortType=zj&city=武汉&internExtend="
result_shixisen =shuju_shixisen(1,browser,url_quanguo)
for i in range(6):
    result_shixisen =zhenghe(result_shixisen,shuju_shixisen(i+2,browser,url_quanguo))
#这里是武汉实习信息版块
browser.get(url_wuhan)
result_shixisen_wuhan =shuju_shixisen(1,browser,'no_url')
for i in range(10):
    try:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        button=browser.find_element_by_xpath('//*[@id="__layout"]/div/div[2]/div[2]/div[1]/div[1]/div[2]/div/ul/li[{}]'.format(i+2))
        button.click()
        time.sleep(1)
        result_shixisen_wuhan =zhenghe(result_shixisen_wuhan,shuju_shixisen(i+2,browser,'no_url'))
    except:
        break
#信息筛选
result_wuhan = [x for x in result_shixisen_wuhan if '武汉' in x[0]]
result_country = result_yingjiesheng + result_shixisen
data_country = pd.DataFrame(result_country,columns = ['base','company','job','url','source','upgrade_time'])
data_wuhan = pd.DataFrame(result_wuhan,columns = ['base','company','job','url','source','upgrade_time'])
print('信息抓取完毕，准备写入数据库')

开始抓取实习信息......



In [7]:
data_country = pd.DataFrame(result_country,columns = ['base','company','job','url','source','upgrade_time'])


In [ ]:
print(data_wuhan)

In [16]:

# 连接database
conn = pymysql.connect(host="localhost", user="root",password="123456",database="shixi",charset="utf8")
# 得到一个可以执行SQL语句的光标对象
cursor = conn.cursor(cursor=pymysql.cursors.DictCursor)
# 定义要执行的SQL语句
sql = "select * from shixi_country;"

# #批量插入数据
# sql_country = "INSERT INTO shixi_country  VALUES (%s, %s, %s, %s, %s,%s)"
# for i in range(len(data_country)):
#     a = data_country['base'][i]
#     b = data_country['company'][i]
#     c = data_country['job'][i]
#     d = data_country['url'][i]
#     e = data_country['source'][i]
#     f = data_country['upgrade_time'][i]
#     values = (a, b, c, d,e,f)#在从也可以进行插入数据格式修改
#     cursor.execute(sql_country,values)

sql_wuhan = "INSERT INTO shixi_wuhan  VALUES (%s, %s, %s, %s, %s,%s)"
for i in range(len(data_wuhan)):
    a = data_wuhan['base'][i]
    b = data_wuhan['company'][i]
    c = data_wuhan['job'][i]
    d = data_wuhan['url'][i]
    e = data_wuhan['source'][i]
    f = data_wuhan['upgrade_time'][i]
    values = (a, b, c, d,e,f)#在从也可以进行插入数据格式修改
    cursor.execute(sql_wuhan,values)

#创建数据表——实习全国
sql3 =" CREATE TABLE IF NOT EXISTS `shixi_wuhan`\
        (\
       `base` VARCHAR(100) NOT NULL,\
       `company` VARCHAR(40) NOT NULL,\
       `job` VARCHAR(100) NOT NULL,\
       `url` VARCHAR(300) NOT NULL,\
       `source` VARCHAR(100) NOT NULL,\
       `upgrade_date` DATE\
        )\
        ENGINE=InnoDB DEFAULT CHARSET=utf8;"


# 执行SQL语句
# cursor.execute(sql3)
result = cursor.fetchmany()
conn.commit()
# 关闭光标对象
cursor.close()
# 关闭数据库连接
conn.close()
print(result)
# print(re)

()


In [ ]:
#接下来学习利用python写接口
import pymysql
import os
import json
import datetime
from flask_cors import *
os.environ['NLS_LANG'] = 'SIMPLIFIED CHINESE_CHINA.UTF8'
from flask import Flask,request
app = Flask(__name__)
class DateEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime.datetime):
            return obj.strftime('%Y-%m-%d %H:%M:%S')

        elif isinstance(obj, datetime.date):
            return obj.strftime("%Y-%m-%d")

        else:
            return json.JSONEncoder.default(self, obj)
@app.route("/api/test",methods=['GET','post'])
def getcontent():
    conn = pymysql.connect(host="localhost", user="root",password="123456",database="shixi",charset="utf8")
    # 得到一个可以执行SQL语句的光标对象
    cursor = conn.cursor(cursor=pymysql.cursors.DictCursor)
    # 定义要执行的SQL语句
    sql1 = "select * from shixi_country;"
    sql2 = "select * from shixi_wuhan;"
    re1 = cursor.execute(sql1)
    result1 = cursor.fetchall()
    re1 = cursor.execute(sql2)
    result2 = cursor.fetchall()
    conn.commit()
    # 关闭光标对象
    cursor.close()
    # 关闭数据库连接
    conn.close()
    data={}
    data_country={}
    data_wuhan={}
    data_country['total']=len(result1)
    data_country['message']=result1
    data_wuhan['total']=len(result2)
    data_wuhan['message']=result2
    data['country']=data_country
    data['wuhan']=data_wuhan
    return json.dumps(data, ensure_ascii=False, indent=4, cls=DateEncoder)
if __name__ == '__main__':
    app.run(host="0.0.0.0", port=8080)
#     app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Aug/2020 19:24:27] "GET /api/test HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2020 19:27:39] "GET /api/test HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2020 19:30:40] "GET /api/test HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2020 19:31:56] "GET /api/test HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2020 19:34:23] "GET /api/test HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2020 19:35:02] "GET /api/test HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2020 19:40:56] "GET /api/test HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2020 19:42:16] "GET /api/test HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2020 19:42:41] "GET /api/test HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2020 19:44:40] "GET /api/test HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2020 19:45:33] "GET /api/test HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2020 19:47:54] "GET /api/test HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2020 19:48:36] "GET /api/test HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2020 19:50:28] "GET /api/test HTTP/1

In [1]:
#内网穿透
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
"""
    ngrok.cc 内网穿透服务 Python 版
    本程序仅适用于ngrok.cc 使用前请先在 https://ngrok.cc 注册账号.
    Linux 系统一般自带Python 可以直接运行
    赋予权限 chmod 755 sunny.py
    直接运行 ./sunny.py --clientid=xxxxxxxxxxxxxxxx
    命令行模式执行 python sunny.py --clientid=xxxxxx 即可运行
    感谢 hauntek 提供的 python-ngrok 原版程序
"""
import getopt
import socket
import ssl
import json
import struct
import random
import sys
import time
import logging
import threading

python_version = sys.version_info >= (3, 0)
if not python_version:
    reload(sys)
    sys.setdefaultencoding('utf8')

options = {
    'clientid':'',
}

def usage():
    print(
        ' -h help \n' \
        ' -a clientid xxxxxxxxxxxxxxxx\n' \
    )
    sys.exit()

try:
    opts, args = getopt.getopt(sys.argv[1:], "h:c:", ['help', "clientid="])
except getopt.GetoptError:
    usage()

if len(opts) == 0:
    print(
        '使用说明\n' \
        '在命令行模式运行 python sunny.py --clientid=xxxxxxxx\n' \
        '如果是多个隧道换成 python sunny.py --clientid=xxxxxxxx,xxxxxxxx\n' \
        '请登录 https://ngrok.cc 获取 clientid\n' \
    )

for option, value in opts:
    if option in ['-h', '--help']:
        usage()
    if option in ['-c', '--clientid']:
        options['clientid'] = value

if options['clientid'] == '':
    if not python_version:
        input_clientid = raw_input('请输入clientid：')
    else:
        input_clientid = str(input('请输入clientid：'))
    if input_clientid != '':
        options['clientid'] = input_clientid
    else:
        sys.exit()

Tunnels = list() # 全局渠道赋值

reqIdaddr = dict()
localaddr = dict()

# ngrok.cc 添加到渠道队列
def ngrok_adds(Tunnel):
    global Tunnels
    for tunnelinfo in Tunnel:
        if tunnelinfo.get('proto'):
            if tunnelinfo.get('proto').get('http'):
                protocol = 'http'
            if tunnelinfo.get('proto').get('https'):
                protocol = 'https'
            if tunnelinfo.get('proto').get('tcp'):
                protocol = 'tcp'

            proto = tunnelinfo['proto'][protocol].split(':') # 127.0.0.1:80 拆分成数组
            if proto[0] == '':
                proto[0] = '127.0.0.1'
            if proto[1] == '' or proto[1] == 0:
                proto[1] = 80

            body = dict()
            body['protocol'] = protocol
            body['hostname'] = tunnelinfo['hostname']
            body['subdomain'] = tunnelinfo['subdomain']
            body['httpauth'] = tunnelinfo['httpauth']
            body['rport'] = tunnelinfo['remoteport']
            body['lhost'] = str(proto[0])
            body['lport'] = int(proto[1])
            Tunnels.append(body) # 加入渠道队列

# ngrok.cc 获取服务器设置
def ngrok_auth(options):
    host = 'www.ngrok.cc'
    port = 443
    try:
        client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        ssl_client = ssl.wrap_socket(client, ssl_version=ssl.PROTOCOL_TLSv1) # ssl.PROTOCOL_TLSv1_2
        ssl_client.connect((host, port))
    except Exception:
        print('连接认证服务器: https://www.ngrok.cc 错误.')
        time.sleep(10)
        sys.exit()

    header = "POST " + "/api/clientid/clientid/%s" + " HTTP/1.1" + "\r\n"
    header += "Content-Type: text/html" + "\r\n"
    header += "Host: %s" + "\r\n"
    header += "\r\n"
    buf = header % (options, host)
    ssl_client.sendall(buf.encode('utf-8')) # 发送请求头

    fd = ssl_client.makefile('rb', 0)
    body = bytes()
    while True:
        line = fd.readline().decode('utf-8')
        if line == "\n" or line == "\r\n":
            chunk_size = int(fd.readline(), 16)
            if chunk_size > 0:
                body = fd.read(chunk_size).decode('utf-8')
                break

    ssl_client.close()

    authData = json.loads(body)
    if authData['status'] != 200:
        print('认证错误:%s, ErrorCode:%s' % (authData['msg'], authData['status']))
        time.sleep(10)
        sys.exit()

    print('认证成功,正在连接服务器...')
    # 设置映射隧道,支持多渠道[客户端id]
    ngrok_adds(authData['data'])
    proto = authData['server'].split(':')
    return proto

print('欢迎使用内网穿透 python-ngrok v1.42\r\nCtrl+C 退出')
serverArr = ngrok_auth(options['clientid'])
host = str(serverArr[0]) # Ngrok服务器地址
port = int(serverArr[1]) # 端口
bufsize = 1024 # 吞吐量

mainsocket = 0

ClientId = ''
pingtime = 0

def getloacladdr(Tunnels, Url):
    protocol = Url[0:Url.find(':')]
    hostname = Url[Url.find('//') + 2:]
    subdomain = hostname[0:hostname.find('.')]
    rport = Url[Url.rfind(':') + 1:]

    for tunnelinfo in Tunnels:
        if tunnelinfo.get('protocol') == protocol:
            if tunnelinfo.get('protocol') in ['http', 'https']:
                if tunnelinfo.get('hostname') == hostname:
                    return tunnelinfo
                if tunnelinfo.get('subdomain') == subdomain:
                    return tunnelinfo
            if tunnelinfo.get('protocol') == 'tcp':
                if tunnelinfo.get('rport') == int(rport):
                    return tunnelinfo

    return dict()

def dnsopen(host):
    try:
        ip = socket.gethostbyname(host)
    except socket.error:
        return False

    return ip

def connectremote(host, port):
    try:
        host = socket.gethostbyname(host)
        client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        ssl_client = ssl.wrap_socket(client, ssl_version=ssl.PROTOCOL_SSLv23)
        ssl_client.connect((host, port))
        ssl_client.setblocking(1)
        logger = logging.getLogger('%s:%d' % ('Conn', ssl_client.fileno()))
        logger.debug('New connection to: %s:%d' % (host, port))
    except socket.error:
        return False

    return ssl_client

def connectlocal(localhost, localport):
    try:
        localhost = socket.gethostbyname(localhost)
        client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client.connect((localhost, localport))
        client.setblocking(1)
        logger = logging.getLogger('%s:%d' % ('Conn', client.fileno()))
        logger.debug('New connection to: %s:%d' % (localhost, localport))
    except socket.error:
        return False

    return client

def NgrokAuth():
    Payload = dict()
    Payload['ClientId'] = ''
    Payload['OS'] = 'darwin'
    Payload['Arch'] = 'amd64'
    Payload['Version'] = '2'
    Payload['MmVersion'] = '2.1'
    Payload['User'] = 'user'
    Payload['Password'] = ''
    body = dict()
    body['Type'] = 'Auth'
    body['Payload'] = Payload
    buffer = json.dumps(body)
    return(buffer)

def ReqTunnel(ReqId, Protocol, Hostname, Subdomain, HttpAuth, RemotePort):
    Payload = dict()
    Payload['ReqId'] = ReqId
    Payload['Protocol'] = Protocol
    Payload['Hostname'] = Hostname
    Payload['Subdomain'] = Subdomain
    Payload['HttpAuth'] = HttpAuth
    Payload['RemotePort'] = RemotePort
    body = dict()
    body['Type'] = 'ReqTunnel'
    body['Payload'] = Payload
    buffer = json.dumps(body)
    return(buffer)

def RegProxy(ClientId):
    Payload = dict()
    Payload['ClientId'] = ClientId
    body = dict()
    body['Type'] = 'RegProxy'
    body['Payload'] = Payload
    buffer = json.dumps(body)
    return(buffer)

def Ping():
    Payload = dict()
    body = dict()
    body['Type'] = 'Ping'
    body['Payload'] = Payload
    buffer = json.dumps(body)
    return(buffer)

def lentobyte(len):
    return struct.pack('<LL', len, 0)

def sendbuf(sock, buf, isblock = False):
    if isblock:
        sock.setblocking(1)
    sock.sendall(buf)
    if isblock:
        sock.setblocking(0)

def sendpack(sock, msg, isblock = False):
    if isblock:
        sock.setblocking(1)
    sock.sendall(lentobyte(len(msg)) + msg.encode('utf-8'))
    logger = logging.getLogger('%s:%d' % ('Send', sock.fileno()))
    logger.debug('Writing message: %s' % msg)
    if isblock:
        sock.setblocking(0)

def tolen(v):
    if len(v) == 8:
        return struct.unpack('<II', v)[0]
    return 0

def getRandChar(length):
    _chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789abcdefghijklmnopqrstuvwxyz"
    return ''.join(random.sample(_chars, length))

# 客户端程序处理过程
def HKClient(sock, linkstate, type, tosock = None):
    global mainsocket
    global ClientId
    global pingtime
    global reqIdaddr
    global localaddr
    recvbuf = bytes()
    while True:
        try:
            if linkstate == 0:
                if type == 1:
                    sendpack(sock, NgrokAuth(), False)
                    linkstate = 1
                if type == 2:
                    sendpack(sock, RegProxy(ClientId), False)
                    linkstate = 1
                if type == 3:
                    linkstate = 1

            recvbut = sock.recv(bufsize)
            if not recvbut: break

            if len(recvbut) > 0:
                if not recvbuf:
                    recvbuf = recvbut
                else:
                    recvbuf += recvbut

            if type == 1 or (type == 2 and linkstate == 1):
                lenbyte = tolen(recvbuf[0:8])
                if len(recvbuf) >= (8 + lenbyte):
                    buf = recvbuf[8:lenbyte + 8].decode('utf-8')
                    logger = logging.getLogger('%s:%d' % ('Recv', sock.fileno()))
                    logger.debug('Reading message with length: %d' % len(buf))
                    logger.debug('Read message: %s' % buf)
                    js = json.loads(buf)
                    if type == 1:
                        if js['Type'] == 'ReqProxy':
                            newsock = connectremote(host, port)
                            if newsock:
                                thread = threading.Thread(target = HKClient, args = (newsock, 0, 2))
                                thread.setDaemon(True)
                                thread.start()
                        if js['Type'] == 'AuthResp':
                            ClientId = js['Payload']['ClientId']
                            logger = logging.getLogger('%s' % 'client')
                            logger.debug('Authenticated with server, client id: %s' % ClientId)
                            sendpack(sock, Ping())
                            pingtime = time.time()
                            for info in Tunnels:
                                reqid = getRandChar(8)
                                sendpack(sock, ReqTunnel(reqid, info['protocol'], info['hostname'], info['subdomain'], info['httpauth'], info['rport']))
                                reqIdaddr[reqid] = (info['lhost'], info['lport'])
                        if js['Type'] == 'NewTunnel':
                            if js['Payload']['Error'] != '':
                                logger = logging.getLogger('%s' % 'client')
                                logger.error('Server failed to allocate tunnel: %s' % js['Payload']['Error'])
                                print('隧道建立失败: %s' % js['Payload']['Error'])
                                time.sleep(30)
                            else:
                                logger = logging.getLogger('%s' % 'client')
                                logger.debug('Tunnel established at %s' % js['Payload']['Url'])
                                print('隧道建立成功: %s' % js['Payload']['Url']) # 注册成功
                                localaddr[js['Payload']['Url']] = reqIdaddr[js['Payload']['ReqId']]
                    if type == 2:
                        if js['Type'] == 'StartProxy':
                            localhost, localport = localaddr[js['Payload']['Url']]

                            newsock = connectlocal(localhost, localport)
                            if newsock:
                                thread = threading.Thread(target = HKClient, args = (newsock, 0, 3, sock))
                                thread.setDaemon(True)
                                thread.start()
                                tosock = newsock
                                linkstate = 2
                            else:
                                body = '<!DOCTYPE html><html><head><meta charset="utf-8"><title>Web服务错误</title><meta name="viewport" content="initial-scale=1,maximum-scale=1,user-scalable=no"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><style>html,body{height:100%%}body{margin:0;padding:0;width:100%%;display:table;font-weight:100;font-family:"Microsoft YaHei",Arial,Helvetica,sans-serif}.container{text-align:center;display:table-cell;vertical-align:middle}.content{border:1px solid #ebccd1;text-align:center;display:inline-block;background-color:#f2dede;color:#a94442;padding:30px}.title{font-size:18px}.copyright{margin-top:30px;text-align:right;color:#000}</style></head><body><div class="container"><div class="content"><div class="title">隧道 %s 无效<br>无法连接到<strong>%s</strong>. 此端口尚未提供Web服务</div></div></div></body></html>'
                                html = body % (js['Payload']['Url'], localhost + ':' + str(localport))
                                header = "HTTP/1.0 502 Bad Gateway" + "\r\n"
                                header += "Content-Type: text/html" + "\r\n"
                                header += "Content-Length: %d" + "\r\n"
                                header += "\r\n" + "%s"
                                buf = header % (len(html.encode('utf-8')), html)
                                sendbuf(sock, buf.encode('utf-8'))

                    if len(recvbuf) == (8 + lenbyte):
                        recvbuf = bytes()
                    else:
                        recvbuf = recvbuf[8 + lenbyte:]

            if type == 3 or (type == 2 and linkstate == 2):
                sendbuf(tosock, recvbuf)
                recvbuf = bytes()

        except socket.error:
            break

    if type == 1:
        mainsocket = False
    if type == 3:
        try:
            tosock.shutdown(socket.SHUT_WR)
        except socket.error:
            tosock.close()

    logger = logging.getLogger('%s:%d' % ('Close', sock.fileno()))
    logger.debug('Closing')
    sock.close()

# 客户端程序初始化
if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO, format='[%(asctime)s] [%(levelname)s] [%(name)s] %(message)s', datefmt='%Y/%m/%d %H:%M:%S')
    logger = logging.getLogger('%s' % 'client')
    logger.debug('python-ngrok v1.5')
    while True:
        try:
            # 检测控制连接是否连接.
            if mainsocket == False:
                ip = dnsopen(host)
                if ip == False:
                    logger = logging.getLogger('%s' % 'client')
                    logger.debug('update dns')
                    print('连接ngrok服务器失败.')
                    time.sleep(10)
                    continue
                mainsocket = connectremote(ip, port)
                if mainsocket == False:
                    logger = logging.getLogger('%s' % 'client')
                    logger.debug('connect failed...!')
                    print('连接ngrok服务器失败.')
                    time.sleep(10)
                    continue
                thread = threading.Thread(target = HKClient, args = (mainsocket, 0, 1))
                thread.setDaemon(True)
                thread.start()

            # 发送心跳
            if pingtime + 20 < time.time() and pingtime != 0:
                sendpack(mainsocket, Ping())
                pingtime = time.time()

            time.sleep(1)

        except socket.error:
            pingtime = 0
        except KeyboardInterrupt:
            sys.exit()

 -h help 
 -a clientid xxxxxxxxxxxxxxxx



SystemExit: 

c:\users\梨苏橙\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
